In [188]:
!pip install striprtf


In [189]:
from striprtf.striprtf import rtf_to_text

with open('/content/algoparams_from_ui1.json.rtf', 'r') as file:
    rtf_text = file.read()

plain_text = rtf_to_text(rtf_text)

print(plain_text)

{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {

      "session_info" : {
        "project_id": "1",
        "experiment_id": "kkkk-11",
        "dataset":"iris_modified.csv",
        "session_name": "test",
        "session_description": "test"
        },

      "target": {
        "prediction_type": "Classification",
        "target": "species",
        "type":"classifiation",
        "partitioning": true
      },
      "train": {
        "policy": "Split the dataset",
        "time_variable": "sepal_length",
        "sampling_method": "No sampling(whole data)",
        "split": "Randomly",
        "k_fold": false,
        "train_ratio": 0.8,
        "random_seed": 10
      },
      "feature_handling": {
        "sepal_length": {
          "feature_name": "sepal_length",
          "is_selected": true,
          "feature_variable_type": "numerical",
          "feature_details": {
            "numerical_handling": "Keep as regular numerical f

In [190]:
# prompt: export plain text variable as a text file

with open('/content/algoparams_from_ui1.txt', 'w') as file:
    file.write(plain_text)


In [191]:
import json
import pandas as pd

# Step 1: Read the CSV file
file_path = "/content/iris_modified.csv"
data = pd.read_csv(file_path)
print("CSV file content:")
print(data.head())



CSV file content:
   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [192]:


import json

# Load the JSON data from the text file
with open("/content/algoparams_from_ui1.txt", "r") as file:
    json_data = json.load(file)

# Extract values from session_info dictionary
session_info = json_data['design_state_data']['session_info']
session_name = session_info['session_name']
session_description = session_info['session_description']
project_id = session_info['project_id']
experiment_id = session_info['experiment_id']
dataset = session_info['dataset']

# Display the extracted values
print("Session Info:")
print(f"Session Name: {session_name}")
print(f"Session Description: {session_description}")
print(f"Project ID: {project_id}")
print(f"Experiment ID: {experiment_id}")
print(f"Dataset: {dataset}")


Session Info:
Session Name: test
Session Description: test
Project ID: 1
Experiment ID: kkkk-11
Dataset: iris_modified.csv


In [193]:
# Extracting information from the 'target' dictionary
target_info = json_data['design_state_data']['target']
target_variable = target_info['target']
prediction_type = target_info['prediction_type']
task_type = target_info['type']

# Determine if it's a classification or regression task
task_type_label = "Classification" if task_type.lower() == "classification" else "Regression"

# Display the extracted information
print("\nTarget Information:")
print(f"Target Variable: {target_variable}")
print(f"Prediction Type: {prediction_type}")
print(f"Task Type: {task_type_label}")



Target Information:
Target Variable: species
Prediction Type: Classification
Task Type: Regression


In [194]:
# Extract the feature_handling dictionary
feature_handling = json_data['design_state_data']['feature_handling']

# Print the extracted feature_handling dictionary
print(feature_handling)

{'sepal_length': {'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values'}}, 'sepal_width': {'feature_name': 'sepal_width', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values'}}, 'petal_length': {'feature_name': 'petal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values'}}, 'petal_width': {'feature_name': 'petal_width', 'is_selected': False,

In [195]:
from sklearn.preprocessing import MinMaxScaler

# Iterate over each feature
for feature_name, details in feature_handling.items():
    if not details.get('is_selected', False):
        # Remove the column from the dataframe if not selected
        data.drop(columns=[feature_name], inplace=True)
        continue

    feature_type = details.get('feature_variable_type', None)

    if feature_type == 'numerical':
        numerical_handling = details['feature_details'].get('numerical_handling', None)
        if numerical_handling == 'Keep as regular numerical feature':
            # No special handling required for numerical features
            pass
        elif numerical_handling == 'Rescale':
          # Initialize MinMaxScaler
            scaler = MinMaxScaler()

          # Rescale the column
            data[feature_name] = scaler.fit_transform(data[[feature_name]])


        elif numerical_handling == 'Impute':
            # Impute missing values for numerical features
            impute_with = details['feature_details'].get('impute_with', None)
            if impute_with == 'Average of values':
                data[feature_name].fillna(data[feature_name].mean(), inplace=True)

    elif feature_type == 'text':
        # Handle text features
        text_handling = details['feature_details'].get('text_handling', None)
        if text_handling == 'Tokenize and hash':
            # Add code for tokenization and hashing if needed
            pass

# Display the modified data
print(data.head())


   sepal_length  sepal_width  petal_length      species
0           5.1          3.5           1.4  Iris-setosa
1           4.9          3.0           1.4  Iris-setosa
2           4.7          3.2           1.3  Iris-setosa
3           4.6          3.1           1.5  Iris-setosa
4           5.0          3.6           1.4  Iris-setosa


In [196]:
from sklearn.model_selection import train_test_split

# Extract parameters for splitting
split_policy = json_data['design_state_data']['train']['policy']
time_variable = json_data['design_state_data']['train']['time_variable']
sampling_method = json_data['design_state_data']['train']['sampling_method']
split_method = json_data['design_state_data']['train']['split']
k_fold = json_data['design_state_data']['train']['k_fold']
train_ratio = json_data['design_state_data']['train'].get('train_ratio', 0.8)
random_seed = json_data['design_state_data']['train'].get('random_seed', None)

# Perform dataset splitting based on the specified policy
if split_policy == 'Split the dataset':
    # Check if the split method is random or time-based
    if split_method == 'Randomly':
        train_data, test_data = train_test_split(data, train_size=train_ratio, random_state=random_seed)
        print("Dataset split randomly with train_ratio =", train_ratio, "and random_seed =", random_seed)
    elif split_method == 'Time-based':
        sorted_data = data.sort_values(by=time_variable)
        split_index = int(len(sorted_data) * train_ratio)
        train_data = sorted_data[:split_index]
        test_data = sorted_data[split_index:]
        print("Dataset split based on time variable =", time_variable, "with train_ratio =", train_ratio)
    else:
        print("Invalid split method specified in the JSON file.")
else:
    print("Invalid splitting policy specified in the JSON file.")

# Print the first few rows of train and test data for verification
print("\nTrain Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())


# Assuming 'target' is the target column name (adjust as needed)
X_train = train_data.drop(target_variable, axis=1)
y_train = train_data[target_variable]

X_test = test_data.drop(target_variable, axis=1)
y_test = test_data[target_variable]


Dataset split randomly with train_ratio = 0.8 and random_seed = 10

Train Data:
     sepal_length  sepal_width  petal_length          species
58            6.6          2.9           4.6  Iris-versicolor
97            6.2          2.9           4.3  Iris-versicolor
129           7.2          3.0           5.8   Iris-virginica
114           5.8          2.8           5.1   Iris-virginica
146           6.3          2.5           5.0   Iris-virginica

Test Data:
     sepal_length  sepal_width  petal_length          species
87            6.3          2.3           4.4  Iris-versicolor
111           6.4          2.7           5.3   Iris-virginica
10            5.4          3.7           1.5      Iris-setosa
91            6.1          3.0           4.6  Iris-versicolor
49            5.0          3.3           1.4      Iris-setosa


In [197]:
# Extract the feature_handling dictionary
algorithms = json_data['design_state_data']['algorithms']

# Print the extracted feature_handling dictionary
print(algorithms)

{'RandomForestClassifier': {'model_name': 'Random Forest Classifier', 'is_selected': True, 'min_trees': 10, 'max_trees': 30, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 30, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 50, 'parallelism': 0}, 'RandomForestRegressor': {'model_name': 'Random Forest Regressor', 'is_selected': False, 'min_trees': 10, 'max_trees': 20, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 25, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 10, 'parallelism': 0}, 'LinearRegression': {'model_name': 'LinearRegression', 'is_selected': False, 'parallelism': 2, 'min_iter': 30, 'max_iter': 50, 'min_regparam': 0.5, 'max_regparam': 0.8, 'min_elasticnet': 0.5, 'max_elasticnet': 0.8}, 'LogisticRegression': {'model_name': 'LogisticRegression', 'is_selected': False, 'parallelism': 2, 'min_iter': 30, 'max_iter': 50, 'min_regparam': 0.5, 'max_regparam': 0.8, 'min_elasticnet': 0.5, 'max

In [198]:
rdh = json_data['design_state_data']['algorithms']['RandomForestClassifier']

# Print the extracted feature_handling dictionary
print(rdh)

{'model_name': 'Random Forest Classifier', 'is_selected': True, 'min_trees': 10, 'max_trees': 30, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 30, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 50, 'parallelism': 0}


In [199]:
from sklearn.metrics import confusion_matrix, classification_report

def compute_classification_metrics(predictor, X_test, y_test):
    # Make predictions
      predicted_labels = predictor.predict(X_test)

    # Compute confusion matrix
      cm = confusion_matrix(y_test, predicted_labels)

    # Compute classification report
      cr = classification_report(y_test, predicted_labels)

      print(cm)
      print(cr)

In [200]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def compute_regression_metrics(predictor, X_test, y_test):
    # Make predictions
    predicted_values = predictor.predict(X_test)

    # Compute R-squared
    r_squared = r2_score(y_test, predicted_values)

    # Compute Adjusted R-squared
    n = len(y_test)
    p = X_test.shape[1]
    adj_r_squared = 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))

    # Compute RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
    print(predictor)


    print("R-squared:", r_squared)
    print("Adjusted R-squared:", adj_r_squared)
    print("RMSE:", rmse)





In [201]:
from sklearn.linear_model import LinearRegression

def build_linear_regression_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        min_iter = params_dict.get('min_iter', 30)
        max_iter = params_dict.get('max_iter', 50)
        min_regparam = params_dict.get('min_regparam', 0.5)
        max_regparam = params_dict.get('max_regparam', 0.8)
        min_elasticnet = params_dict.get('min_elasticnet', 0.5)
        max_elasticnet = params_dict.get('max_elasticnet', 0.8)

        # Initialize model with hyperparameters
        model = LinearRegression(n_jobs=params_dict.get('parallelism', 1))

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model







In [202]:
from sklearn.linear_model import LogisticRegression

def build_logistic_regression_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        min_iter = params_dict.get('min_iter', 30)
        max_iter = params_dict.get('max_iter', 50)
        min_regparam = params_dict.get('min_regparam', 0.5)
        max_regparam = params_dict.get('max_regparam', 0.8)
        min_elasticnet = params_dict.get('min_elasticnet', 0.5)
        max_elasticnet = params_dict.get('max_elasticnet', 0.8)

        # Initialize model with hyperparameters
        model = LogisticRegression(n_jobs=params_dict.get('parallelism', 1))

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model




In [203]:
from sklearn.linear_model import Ridge

def build_ridge_regression_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        regularization_term = params_dict.get('regularization_term', 1.0)
        min_iter = params_dict.get('min_iter', 30)
        max_iter = params_dict.get('max_iter', 50)

        # Initialize model with hyperparameters
        model = Ridge(alpha=regularization_term,
                      max_iter=max_iter,
                      min_iter=min_iter)

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model





In [204]:
from sklearn.linear_model import Lasso

def build_lasso_regression_model(params_dict, X_train, X_test, y_train, y_test, prediction_type):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        regularization_term = params_dict.get('regularization_term', 1.0)
        min_iter = params_dict.get('min_iter', 30)
        max_iter = params_dict.get('max_iter', 50)

        # Initialize model with hyperparameters
        model = Lasso(alpha=regularization_term,
                      max_iter=max_iter,
                      min_iter=min_iter)

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model




In [205]:
from sklearn.linear_model import ElasticNet

def build_elasticnet_regression_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        regularization_term = params_dict.get('regularization_term', 1.0)
        min_iter = params_dict.get('min_iter', 30)
        max_iter = params_dict.get('max_iter', 50)
        min_regparam = params_dict.get('min_regparam', 0.5)
        max_regparam = params_dict.get('max_regparam', 0.8)
        min_elasticnet = params_dict.get('min_elasticnet', 0.5)
        max_elasticnet = params_dict.get('max_elasticnet', 0.8)

        # Initialize model with hyperparameters
        model = ElasticNet(alpha=regularization_term,
                           l1_ratio=params_dict.get('min_elasticnet', 0.5),
                           max_iter=max_iter,
                           min_iter=min_iter)

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model



In [206]:
from sklearn.ensemble import RandomForestClassifier

def build_model(params_dict, X_train, X_test, y_train, y_test):
    # Extract hyperparameters
    min_trees = params_dict.get('min_trees', 10)
    max_trees = params_dict.get('max_trees', 30)
    min_depth = params_dict.get('min_depth', None)
    max_depth = params_dict.get('max_depth', None)
    min_samples_per_leaf_min_value = params_dict.get('min_samples_per_leaf_min_value', 5)
    parallelism = params_dict.get('parallelism', 0)

    # Initialize model with hyperparameters
    model = RandomForestClassifier(n_estimators=min_trees,

                                   max_depth=max_depth,
                                   min_samples_leaf=min_samples_per_leaf_min_value,
                                   )

    # Train the model
    model.fit(X_train, y_train)
    if prediction_type=="Classification":
      compute_classification_metrics(model,X_test, y_test)
    else:
      compute_regression_metrics(model, X_test, y_test)
    return model



In [207]:
from sklearn.ensemble import RandomForestRegressor


def regress(params_dict, X_train, y_train):
    # Extract hyperparameters
    min_trees = params_dict.get('min_trees', 10)
    max_trees = params_dict.get('max_trees', 20)
    min_depth = params_dict.get('min_depth', None)
    max_depth = params_dict.get('max_depth', None)
    min_samples_per_leaf_min_value = params_dict.get('min_samples_per_leaf_min_value', 5)
    min_samples_per_leaf_max_value = params_dict.get('min_samples_per_leaf_max_value', 10)
    parallelism = params_dict.get('parallelism', 0)

    # Initialize model with hyperparameters
    model = RandomForestRegressor(n_estimators=min_trees,
                                   max_depth=max_depth,
                                   min_samples_leaf=min_samples_per_leaf_min_value,
                                   )

    # Train the model
    model.fit(X_train, y_train)
    if prediction_type=="Classification":
      compute_classification_metrics(model,X_test, y_test)
    else:
      compute_regression_metrics(model, X_test, y_test)

    return model

In [208]:
import xgboost as xgb

def build_xgboost_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        max_depth_of_tree = params_dict.get('max_depth_of_tree', [3, 6])
        learning_rate = params_dict.get('learningRate', [0.1, 0.3])
        l1_regularization = params_dict.get('l1_regularization', [0])
        l2_regularization = params_dict.get('l2_regularization', [0])
        gamma = params_dict.get('gamma', [0])
        min_child_weight = params_dict.get('min_child_weight', [1])
        sub_sample = params_dict.get('sub_sample', [1])
        col_sample_by_tree = params_dict.get('col_sample_by_tree', [1])

        # Initialize model with hyperparameters
        model = xgb.XGBRegressor(max_depth=max_depth_of_tree,
                                  learning_rate=learning_rate,
                                  reg_alpha=l1_regularization,
                                  reg_lambda=l2_regularization,
                                  gamma=gamma,
                                  min_child_weight=min_child_weight,
                                  subsample=sub_sample,
                                  colsample_bytree=col_sample_by_tree,
                                  objective='reg:squarederror',
                                  random_state=params_dict.get('random_state', 0))

        # Train the model
        eval_set = [(X_train, y_train), (X_test, y_test)]
        model.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set,
                  early_stopping_rounds=params_dict.get('early_stopping_rounds', 2))

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model



In [209]:
from sklearn.tree import DecisionTreeRegressor

def build_decision_tree_regressor_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        min_depth = params_dict.get('min_depth', 4)
        max_depth = params_dict.get('max_depth', 7)
        min_samples_per_leaf = params_dict.get('min_samples_per_leaf', [12, 6])
        criterion = 'entropy' if params_dict.get('use_entropy', True) else 'gini'
        splitter = 'best' if params_dict.get('use_best', True) else 'random'

        # Initialize model with hyperparameters
        model = DecisionTreeRegressor(min_samples_leaf=min_samples_per_leaf,
                                       max_depth=max_depth,
                                       min_samples_split=min_samples_per_leaf,
                                       criterion=criterion,
                                       splitter=splitter,
                                       random_state=params_dict.get('random_state', None))

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model




In [210]:
from sklearn.svm import SVC

def build_svm_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        c_values = params_dict.get('c_value', [1.0])
        kernels = []
        if params_dict.get('linear_kernel', True):
            kernels.append('linear')
        if params_dict.get('rep_kernel', True):
            kernels.append('rbf')
        if params_dict.get('polynomial_kernel', True):
            kernels.append('poly')
        if params_dict.get('sigmoid_kernel', True):
            kernels.append('sigmoid')
        auto = params_dict.get('auto', True)
        scale = params_dict.get('scale', True)
        custom_gamma_values = params_dict.get('custom_gamma_values', True)
        tolerance = params_dict.get('tolerance', 0.001)
        max_iterations = params_dict.get('max_iterations', -1)

        # Initialize list to store models
        models = []

        # Train a model for each combination of C value and kernel
        for c in c_values:
            for kernel in kernels:
                # Initialize model with hyperparameters
                model = SVC(C=c, kernel=kernel, probability=True, tol=tolerance, max_iter=max_iterations,
                            decision_function_shape='ovr', random_state=params_dict.get('random_state', None))

                # Train the model
                model.fit(X_train, y_train)

                models.append(model)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model


In [211]:
from sklearn.neighbors import KNeighborsClassifier

def build_knn_model(params_dict, X_train, X_test, y_train, y_test, prediction_type):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        k_values = params_dict.get('k_value', [5])
        distance_weighting = params_dict.get('distance_weighting', False)
        neighbour_finding_algorithm = params_dict.get('neighbour_finding_algorithm', 'auto')
        p_value = params_dict.get('p_value', 2)

        # Initialize list to store models
        models = []

        # Train a model for each value of k
        for k in k_values:
            # Initialize model with hyperparameters
            model = KNeighborsClassifier(n_neighbors=k,
                                         weights='distance' if distance_weighting else 'uniform',
                                         algorithm=neighbour_finding_algorithm,
                                         p=p_value)

            # Train the model
            model.fit(X_train, y_train)

            models.append(model)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model




In [212]:
from sklearn.neural_network import MLPClassifier

def build_neural_network_model(params_dict, X_train, X_test, y_train, y_test, prediction_type):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        hidden_layer_sizes = params_dict.get('hidden_layer_sizes', [100])
        activation = params_dict.get('activation', 'relu')
        alpha_value = params_dict.get('alpha_value', 0.0001)
        max_iterations = params_dict.get('max_iterations', 200)
        convergence_tolerance = params_dict.get('convergence_tolerance', 1e-4)
        early_stopping = params_dict.get('early_stopping', False)
        solver = params_dict.get('solver', 'adam')
        shuffle_data = params_dict.get('shuffle_data', True)
        initial_learning_rate = params_dict.get('initial_learning_rate', 0.001)
        automatic_batching = params_dict.get('automatic_batching', False)
        beta_1 = params_dict.get('beta_1', 0.9)
        beta_2 = params_dict.get('beta_2', 0.999)
        epsilon = params_dict.get('epsilon', 1e-8)
        power_t = params_dict.get('power_t', 0.5)
        momentum = params_dict.get('momentum', 0.9)
        use_nesterov_momentum = params_dict.get('use_nesterov_momentum', False)

        # Initialize model with hyperparameters
        model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                              activation=activation,
                              alpha=alpha_value,
                              max_iter=max_iterations,
                              tol=convergence_tolerance,
                              early_stopping=early_stopping,
                              solver=solver,
                              shuffle=shuffle_data,
                              learning_rate_init=initial_learning_rate,
                              batch_size='auto' if automatic_batching else 'auto',
                              beta_1=beta_1,
                              beta_2=beta_2,
                              epsilon=epsilon,
                              power_t=power_t,
                              momentum=momentum,
                              nesterovs_momentum=use_nesterov_momentum,
                              random_state=params_dict.get('random_state', None))

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model





In [213]:
from sklearn.tree import DecisionTreeClassifier

def build_decision_tree_classifier_model(params_dict, X_train, X_test, y_train, y_test):
    if params_dict.get('is_selected', False):
        # Extract hyperparameters
        min_depth = params_dict.get('min_depth', 4)
        max_depth = params_dict.get('max_depth', 7)
        min_samples_per_leaf = params_dict.get('min_samples_per_leaf', [12, 6])
        criterion = 'entropy' if params_dict.get('use_entropy', True) else 'gini'
        splitter = 'best' if params_dict.get('use_best', True) else 'random'

        # Initialize model with hyperparameters
        model = DecisionTreeClassifier(max_depth=max_depth,
                                       min_samples_split=2,
                                       criterion=criterion,
                                       splitter=splitter,
                                       random_state=params_dict.get('random_state', None))

        # Train the model
        model.fit(X_train, y_train)

        if prediction_type=="Classification":
          compute_classification_metrics(model,X_test, y_test)
        else:
          compute_regression_metrics(model, X_test, y_test)
        return model




In [214]:
for model_name, model_params in algorithms.items():
  print(model_name, model_params)

RandomForestClassifier {'model_name': 'Random Forest Classifier', 'is_selected': True, 'min_trees': 10, 'max_trees': 30, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 30, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 50, 'parallelism': 0}
RandomForestRegressor {'model_name': 'Random Forest Regressor', 'is_selected': False, 'min_trees': 10, 'max_trees': 20, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 25, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 10, 'parallelism': 0}
LinearRegression {'model_name': 'LinearRegression', 'is_selected': False, 'parallelism': 2, 'min_iter': 30, 'max_iter': 50, 'min_regparam': 0.5, 'max_regparam': 0.8, 'min_elasticnet': 0.5, 'max_elasticnet': 0.8}
LogisticRegression {'model_name': 'LogisticRegression', 'is_selected': False, 'parallelism': 2, 'min_iter': 30, 'max_iter': 50, 'min_regparam': 0.5, 'max_regparam': 0.8, 'min_elasticnet': 0.5, 'max_elasticnet': 0.

In [217]:
from sklearn.metrics import accuracy_score, mean_squared_error

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Evaluate on training set
    train_predictions = model.predict(X_train)
    train_score = model.score(X_train, y_train)  # You can use any appropriate metric here

    # Evaluate on testing set
    test_predictions = model.predict(X_test)
    test_score = model.score(X_test, y_test)  # You can use any appropriate metric here

    return train_score, test_score

def call_model_functions(algorithms, X_train, X_test, y_train, y_test):
    model_build_functions = {
        "RandomForestClassifier": build_model,
        "RandomForestRegressor": regress,
        "LinearRegression": build_linear_regression_model,
        "LogisticRegression": build_logistic_regression_model,
        "RidgeRegression": build_ridge_regression_model,
        "LassoRegression": build_lasso_regression_model,
        "ElasticNetRegression": build_elasticnet_regression_model,
        "xg_boost": build_xgboost_model,
        "DecisionTreeRegressor": build_decision_tree_regressor_model,
        "DecisionTreeClassifier": build_decision_tree_classifier_model,
        "SVM": build_svm_model,
        "KNN": build_knn_model,
        "neural_network": build_neural_network_model,
    }

    best_model = None
    best_model_name = None
    best_test_score = -float('inf')

    for model_name, model_params in algorithms.items():
        if model_name in model_build_functions:
            model_build_function = model_build_functions[model_name]
            if model_params.get('is_selected', False):
                model = model_build_function(model_params, X_train, X_test, y_train, y_test)
                train_score, test_score = evaluate_model(model, X_train, X_test, y_train, y_test)
                print(f"{model_name} Train Score:", train_score)
                print(f"{model_name} Test Score:", test_score)
                if test_score > best_test_score:
                    best_test_score = test_score
                    best_model = model
                    best_model_name = model_name
        else:
            print(f"Sorry, {model_name} is not a valid model name.")

    if best_model:
        print("Model Evaluation Report")
        print("-----------------------")
        print(f"Best Model: {best_model_name}")
        return best_model
    else:
        print("No models were selected.")

# Example usage:
best_model = call_model_functions(algorithms, X_train, X_test, y_train, y_test)


[[10  0  0]
 [ 0 11  2]
 [ 0  0  7]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      0.85      0.92        13
 Iris-virginica       0.78      1.00      0.88         7

       accuracy                           0.93        30
      macro avg       0.93      0.95      0.93        30
   weighted avg       0.95      0.93      0.93        30

RandomForestClassifier Train Score: 0.9583333333333334
RandomForestClassifier Test Score: 0.9333333333333333
[[10  0  0]
 [ 0 12  1]
 [ 0  0  7]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.88      1.00      0.93         7

       accuracy                           0.97        30
      macro avg       0.96      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30

DecisionTreeC